In [1]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(15,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot
def REC(y_true , y_pred):
    
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):
            y_true[j]=float(y_true[j])
            y_true[j]
            y_pred[j]
            
            np.linalg.norm(y_true[j] - y_pred[j])
            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC
def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict

C:\Users\Liuya\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
KU = pd.read_csv('KU_dataset_2.csv')
KU.fillna(0.0, inplace=True)
KU = KU.drop(['Userid'], axis=1)
KU.head(3)

,Add_Bookmark,Add_Marker,Add_Memo,Change_Memo,Close,Delete_Bookmark,Delete_Marker,Delete_Memo,Jump,Next,Open,Prev,Search,Marker,Memo,Score
0,0,0,1,0,2,0,0,0,1,281,5,80,0,0,2,100.0
1,0,0,0,0,12,0,0,0,7,528,13,171,0,0,0,62.5
2,1,1,0,0,23,1,1,0,4,160,26,14,0,2,0,87.5


In [3]:
data=KU
data.to_csv('KU_data2.csv')
data_y= data['Score']
data_x = data.drop(['Score'], axis=1)

In [4]:
data

,Add_Bookmark,Add_Marker,Add_Memo,Change_Memo,Close,Delete_Bookmark,Delete_Marker,Delete_Memo,Jump,Next,Open,Prev,Search,Marker,Memo,Score
0,0,0,1,0,2,0,0,0,1,281,5,80,0,0,2,100.0
1,0,0,0,0,12,0,0,0,7,528,13,171,0,0,0,62.5
2,1,1,0,0,23,1,1,0,4,160,26,14,0,2,0,87.5
3,1,0,0,0,6,1,0,0,2,84,9,19,0,0,0,18.8
4,2,1,0,0,1,1,0,0,3,655,6,398,0,1,0,81.3
5,0,0,0,0,8,0,0,0,2,308,8,99,0,0,0,87.5
6,1,0,0,0,0,1,0,0,13,463,17,239,0,0,0,87.5
7,2,3,4,0,3,1,3,0,8,656,11,374,0,6,9,93.8
8,0,3,3,4,11,0,1,0,16,768,15,275,0,4,7,100.0
9,3,0,0,0,9,3,0,0,12,1387,22,867,0,0,0,68.8


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [6]:
mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)

In [7]:
mlp.fit(X_train, y_train) 
result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)
print(result_dict)

{'rmse': 21.66544441045311, 'mape': 42.053660398395046, 'r2': 0.005301496661103244, 'auc': 0.8782352941176471, 'training_loss': 18.88327140126103, 'f_test': 21.413134247581382, 'p': '***', 'y_pred': array([57.70491 , 70.216866, 69.93491 , 71.23035 , 70.47684 , 74.00982 ,
       68.518814, 70.01591 , 69.7988  , 81.81362 , 70.178955, 69.89348 ,
       70.021515, 70.3578  , 71.33486 , 70.46225 , 67.70157 ],
      dtype=float32), 'y_validate': array([70.02587 , 69.779526, 70.42195 , 69.684135, 70.57195 , 69.89697 ,
       69.296875, 70.16338 , 69.78199 , 71.68457 , 70.863846, 65.279274,
       69.8042  , 69.815544, 70.63162 , 68.40521 , 69.57694 , 69.9798  ,
       70.41649 , 62.972694, 70.62549 , 69.92788 , 70.031166, 69.65449 ,
       70.31808 , 70.278915, 62.605663, 70.43397 , 62.202568, 70.95006 ,
       70.69347 , 62.354992, 70.53189 , 69.913284, 67.08332 , 72.71093 ,
       70.49358 , 70.03841 ], dtype=float32)}
